# Python code of Gibbs sampling model for family test result Covid problem

In [1]:
# import neccessary libraries
import numpy as np
import random
import sympy as sym
from joblib import Parallel, delayed
from sympy import *
from sympy import symbols
from sympy.plotting import plot, plot3d
from datetime import datetime
from scipy.stats import beta
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

In [2]:
# function to count the number of the infected neighbores of i at t:
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [3]:
# Function to obtain the very initial sample of X, using forwad sampling:
def Forward_Sampling(T,n,G,F,param):
    
    alpha_=param[0]
    bata_=param[1]
    bataf=param[2]
    gama_=param[3]
    p0=P
    p1=1-P
    x=int((1-P)*n)
    X=np.zeros((n,T))  
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    infR=CNbr(G,X,n,T)
    for t in range(T-1):
        cf=F.dot(X.T[t])
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=(1-alpha_-beta_*c-betaf*cf[i])
                p1=(alpha_+beta_*c+betaf*cf[i])
            else:
                p0=(gama_)
                p1=(1-gama_)
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 
    return X

In [4]:
def Sample_hidden_state(X,G,F,unique_rows,Y,param,t):
    
    alpha_=param[0]
    beta_=param[1]
    bataf=param[2]
    gama_=param[3]
    theta_0_=param[3]
    thata_1_=param[4]
    
    
    number_of_infected_members_in_family=F.dot(X.T[t])
    
    if (t==0):
        c=G[t].dot(X.T[t])
    else:    
        c=G[t-1].dot(X.T[t-1])
        
    for i in range(n):
        p_0,p_1=0,0
        pow_stay_healthy,pow_become_infected,pow_recovery,pow_stay_infected=0,0,0,0
        pow0=np.count_nonzero(Y[i,t] == 0)  
        pow1=np.count_nonzero(Y[i,t] == 1)
        number_of_members_in_family=np.sum(family_index(i,unique_rows))
        if (X[i,t]==1):
            if t==0:
                c0=c[i]-1
                c1=c[i]
            else:
                c0=c[i]
                c1=c[i]
            number_of_infected_members_in_family0=number_of_infected_members_in_family[i]-1
            number_of_infected_members_in_family1=number_of_infected_members_in_family[i]
        else:
            
            c0=c[i]
            c1=c[i]+1
            number_of_infected_members_in_family0=number_of_infected_members_in_family[i]
            number_of_infected_members_in_family1=number_of_infected_members_in_family[i]+1
        number_of_healthy_members_in_family0=number_of_members_in_family-number_of_infected_members_in_family0
        number_of_healthy_members_in_family1=number_of_members_in_family-number_of_infected_members_in_family1
        
        p_0=((1-(1-theta_0_)**number_of_healthy_members_in_family0)*theta_1_**number_of_infected_members_in_family0)**pow0*((1-(1-theta_1_)**number_of_infected_members_in_family0)*theta_0_**number_of_healthy_members_in_family0)**pow1
        
        p_1=((1-(1-theta_0_)**number_of_healthy_members_in_family1)*theta_1_**number_of_infected_members_in_family1)**pow0*((1-(1-theta_1_)**number_of_infected_members_in_family1)*theta_0_**number_of_healthy_members_in_family1)**pow1
        
        stay_healthy0= 1-alpha_-beta_*c0-betaf*number_of_infected_members_in_family0
        become_infected0=alpha_+beta_*c0+betaf*number_of_infected_members_in_family0
    
        stay_healthy1=1-alpha_-beta_*c1-betaf*number_of_infected_members_in_family1
        become_infected1=alpha_+beta_*c1+betaf*number_of_infected_members_in_family1
        
        if t==T-1:
            if X[i,t-1]==0:
                p0=p_0*stay_healthy0
                p1=p_1*become_infected0
            else:
                p0=p_0*gama_
                p1=p_1*(1-gama_) 
        else:        
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[0][i],state_transition)
    
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
    
            p0=P*p_0*(stay_healthy0)**pow_stay_healthy*(become_infected0)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            p1=(1-P)*p_1*(stay_healthy1)**pow_stay_healthy*(become_infected1)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            
        if p0+p1==0:
            l=0.5
        else:
            l=p1/(p0+p1)
        
        X[i,0]=np.random.binomial( 1,  l,size=None)    
        
    return X    

In [5]:
# Gibbs sampling to obtain X, as new sample of posterior distribution:
def Calculate_X(K,T,X,G,F1,Y,param):
    for k in range(K):
        for t in range(T):
            X=Sample_hidden_state(X,G,F1,unique_rows,Y,param,t)
    return X                

In [6]:
# function to sample new parameters and update parameters:
def Params(R,G,F,X,n,T,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    bataf=param[2]
    gama_=param[3]
    theta_0_=param[4]
    thata_1_=param[5]    
   
    TP=np.sum(np.multiply(X,Y))
    FP=n*T-np.count_nonzero(X-Y+1)
    
    infR=np.array(CNbr(G,X,n,T))
    print(a_alpha +  np.count_nonzero(R==0) , b_alpha +np.count_nonzero(X==0)- np.count_nonzero(R==0))
    alpha_=Sample_alpha(a_alpha +  np.count_nonzero(R==0) , b_alpha +np.count_nonzero(X==0)- np.count_nonzero(R==0))
    beta_=Sample_beta(a_beta + np.count_nonzero(R==2) , b_beta +np.sum(np.multiply((1-X),infR))-np.count_nonzero(R==2))
    betaf=Sample_betaf(a_betaf + np.count_nonzero(R==3) , b_betaf +np.sum(np.multiply((1-X),F.dot(X.T)))-np.count_nonzero(R==3))
    gama_=Sample_gama(a_gama +(T-1)*n-np.count_nonzero(X[:,:-1]-X[:,1:]-1), b_gama+np.sum(X)-(T-1)*n+np.count_nonzero(X[:,:-1]-X[:,1:]-1))
    theta_0_=Sample_theta0( a_teta0+FP,b_teta0+n*T-np.count_nonzero(X)-FP)
    theta_1_=Sample_theta1( a_teta1+TP,b_teta1+np.count_nonzero(X)-TP)
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infr=int(infR[i,t])
            cf=int(F.dot(X.T[t])[i])
            pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
            pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
            pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                elif (cf!=0):
                    if ((np.equal(1,v[-cf:])).any):
                        R[i,t]=3
                else:    
                    R[i,t]=2
    param=[alpha_,beta_,betaf,gama_,theta_0_,theta_1_]
    return param,R

# Functions to sample from beta distribution


In [7]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=10000):
        print("al",i)
        if (i>0.001)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        


In [8]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        if (i>0.001)&(i<0.0451):
            beta_=round(i,4)
            break
    return beta_        

In [9]:
def Sample_betaf(a_betaf, b_betaf):
    for i in beta.rvs(a_betaf, b_betaf, size=10000):
        if (i>0.001)&(i<0.091):
            betaf=round(i,4)
            break
    return betaf        

In [10]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=10000):
        if (i>0.1)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  

In [11]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        print("th",i)
        if (i>0.1)&(i<0.51):
            theta_0_=round(i,3)
            break
    return theta_0_  

In [12]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=100000):
        if .9901>i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  

In [13]:
# initialize parameters for beta distributions:
a_alpha=1
b_alpha=1
a_beta=1
b_beta=500
a_betaf=1
b_betaf=6
a_gama=1
b_gama=20
a_teta0=1
b_teta0=3
a_teta1=4000
b_teta1=100
P=.5
number_families=30

In [14]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
betaf=Sample_betaf(a_betaf, b_betaf)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

al 0.9156656267085476
al 0.6694385795910821
al 0.012354041725840977
th 0.12369256773179022


In [15]:
params=np.array([alpha_,beta_,betaf,gama_,theta_0_,theta_1_])
params

array([0.012 , 0.0033, 0.0222, 0.238 , 0.124 , 0.977 ])

In [16]:
# funtion to retun related family index of individual i:
def family_index(i,unique_rows):
    for j in range(n):
        if unique_rows[j,i]==1:
            return j

In [17]:
# Function to generates Synthetic dataset
def Synthetic_Data(n,T,y,params,number_families):
    alpha_,beta_,betaf,gama_,theta_0_,theta_1_=params[0],params[1],params[2],params[3],params[4],params[5]
    x=int((1-P)*n)

    X=np.zeros((n,T))
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    # Random social network
    G=[]
    for j in range(T):
        g=np.identity(n,dtype=int)
        for i in range(n):
            inx=np.random.choice(range(i,n), y)
            g[i,inx]=1  
            g[inx,i]=1
        G.append(g)
    G=np.array(G)
    infR=CNbr(G,X,n,T)
    # Synthetize X, using params,G and transition probability:
    for t in range(T-1):
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c
                p1=alpha_+beta_*c
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            if (l<0)|(l>1):
                print(p0,p1,l)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 

    # Synthetize Y, using params,G, X, emission probability:
    Y=np.zeros((n,T))
    for t in range(T):
        for i in range(n):
            if X[i,t]==0:
                Y[i,t]=np.random.binomial( 1, theta_0_,size=None) 
            else:
                Y[i,t]=np.random.binomial( 1, theta_1_,size=None) 
    a=list(range(n))
    Family=np.identity(n,dtype=int)
    for i in range(1,number_families):
        number_family_members=np.random.randint(2, 5)
        inx=random.sample(list(a), number_family_members)
        a=set(a).difference(inx)
        for j in inx:
            for k in inx:
                Family[j,k]=1
    return G,Y,X,Family            

In [18]:
# Generate synthetic data,G ,Y:
n,T,y=100,100,10
synthetic_data=Synthetic_Data(n,T,y,params,number_families)
G,Y,X,Family =synthetic_data[0],synthetic_data[1],synthetic_data[2],synthetic_data[3]

In [19]:
# Save true value of X in Z, as the correct label of data:
Z=X
np.sum(Z)

590.0

In [20]:
F=Family

In [21]:
# synthetize family test result matrix,YF:
unique_rows = np.unique(Family, axis=0)
unique_rows.shape
YF=np.zeros((n,T))
YF=np.dot(unique_rows,Y)
YF.shape

(44, 100)

In [22]:
#W=np.round(np.random.uniform(0.0, .002, (n,1)),3)
G=G-Family+np.identity(n,dtype=int)
G = np.where(G <0, 0, G)
YF = np.where(YF>1, 1, YF)

In [23]:
Y1=Y

In [24]:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]

In [25]:
# define auxiliary variable R(n,t):
infR=np.array(CNbr(G,X,n,T))
R=np.zeros((n,T))+1
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        cf=int(F.dot(X.T[t])[i])
        pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
        pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
        pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
        if (X[i][t]==0)&(X[i][t+1]==1):
            if v[0]==1:
                R[i,t]=0
            elif ((v[-cf:]==1).any())&(cf!=0): 
                R[i,t]=3
            else:    
                R[i,t]=2


In [26]:
# Run Gibbs sampling algorithm to estimate X:
U=10
K=100
X=Forward_Sampling(T,n,G,F,params)
X=Calculate_X(K,T,X,G,F,Y,params)
winsound.Beep(freq, duration)

In [27]:
np.sum(X)

669.0

In [28]:
# define auxiliary variable R(n,t):
infR=np.array(CNbr(G,X,n,T))
R=np.zeros((n,T))+1
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        cf=int(F.dot(X.T[t])[i])
        pr_a=alpha_/(alpha_+beta_*infr+betaf*cf)
        pr_b=beta_/(alpha_+beta_*infr+betaf*cf)
        pr_bf=betaf/(alpha_+beta_*infr+betaf*cf)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr+[pr_bf]*cf)
        if (X[i][t]==0)&(X[i][t+1]==1):
            if v[0]==1:
                R[i,t]=0
            elif (cf!=0):
                if ((v[-cf:]==1).any()):
                    R[i,t]=3
            else:    
                R[i,t]=2


In [29]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    print("Current Parameters:",params)
    prm=Params(R,G,F,X,n,T,Y,params)
    params=prm[0]
    X=Calculate_X(K,T,X,G,F,Y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)            

******************* Iteration: 0 *****************************************************************************
Current Time is : 16:34:18
Current Parameters: [0.012  0.0033 0.0222 0.238  0.124  0.977 ]
104 9229
al 0.01246695867966337
th 0.3996713256001534
Verctor of Health States: 
 [[1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
******************* Iteration: 1 *****************************************************************************
Current Time is : 16:35:22
Current Parameters: [0.012, 0.0025, 0.0132, 0.282, 0.4, 0.907]
105 9229
al 0.01155622023020028
th 0.3960729792797397
Verctor of Health States: 
 [[1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
******************* Iteration: 2 *****************************************************************************
Current Ti

In [30]:
params

[0.012, 0.0019, 0.0149, 0.251, 0.401, 0.905]

In [32]:
# Compare X, Z and estimate accuracy of the model:

In [35]:
print(np.sum(X),np.sum(Y),np.count_nonzero(Y-X))

674.0 4008.0 4072


In [ ]:
# synthetized observation vectore:
Y_=np.zeros((n,T))
for t in range(T):
    for i in range(n):
        if X[i,t]==0:
            Y_[i,t]=np.random.binomial( 1, theta_0_,size=None) 
        else:
            Y_[i,t]=np.random.binomial( 1, theta_1_,size=None) 

In [ ]:
print(np.sum(Y_),np.sum(Y),np.count_nonzero(Y-Y_))

In [ ]:
Y=Y_

# The following is for family test result problem:

In [ ]:
# Synthetize data for famity test result problem

In [ ]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=1000):
        if (i>0.029)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        


In [ ]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        if (i>0.00491)&(i<0.01451):
            beta_=round(i,4)
            break
    return beta_        


In [ ]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=1000):
        if (i>0.291)&(i<0.31):
            gama_=round(i,3)
            break
    return gama_  


In [ ]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        print(i)
        if (i<0.19975):
            theta_0_=round(i,3)
            break
    return theta_0_  


In [ ]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=100000):
        if .9901>i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  


In [ ]:
# initialize parameters for beta distributions:
a_alpha=2
b_alpha=7
a_beta=2
b_beta=10
a_gama=1
b_gama=3
a_teta0=.1
b_teta0=500
a_teta1=4000
b_teta1=100

In [ ]:
# Choose some params close to obtimized params:
alpha_= 0.05
beta_= 0.01
gama_= 0.3
theta_0_= 0.07
theta_1_= 0.8
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

In [ ]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

In [ ]:
params=np.array([alpha_,beta_,gama_,theta_0_,theta_1_])
params

In [ ]:
#Use uptimized params:
number_families=4
alpha_=params[0]
bata_=params[1]
gama_=params[2]
theta_0_=params[3]
thata_1_=params[4]    
p_=0.9

In [ ]:
# Synthetize the "Family" matrix, which denotes that each two individuals are in the same family or not:
a=list(range(n))
Family=np.identity(n,dtype=int)
for i in range(1,number_families):
    number_family_members=np.random.randint(2, 5)
    inx=random.sample(list(a), number_family_members)
    a=set(a).difference(inx)
    for j in inx:
        for k in inx:
            Family[j,k]=1
F=Family
F1=F-np.identity(n, dtype=int)

In [ ]:
# Synthetize G,Y,X:
n,T,y=10,10,2
synthetic_data=Synthetic_Data(n,T,y,params)
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

# Simulation data is ready now, YF,G,Family:

In [ ]:
F

In [ ]:
G[1]

In [ ]:
# ایده اول برای مین فیلد و شبه مشاهدات

In [ ]:
y=np.zeros((n,T))
for i in range(n):
    for j in range(n):
         if unique_rows[j,i]==1:
                break
    for t in range(T):
       
        y[i,t]=int(YF[j,t]-np.dot(np.multiply(unique_rows[j],X.T[t]),W)+X[i,t]*W[i])
        y = np.where(y <0, 0, y)
y 

In [ ]:
params

In [ ]:
# Run Forward sampling algorithm to estimate X:
U=10
J=100
P=p_
X=Forward_Sampling(T,n,G,params)
X=Calculate_X(J,T,n,X,G,y,params)
winsound.Beep(freq, duration)

In [ ]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [ ]:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,y,params)
    params=prm[0]
    print("updated parameters:",params)
    X=Calculate_X(J,T,n,X,G,y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)

In [ ]:
for i in range(n):
    for t in range(T):
        if X[i,t]==0:
            yy[i,t]=np.random.binomial( 1, theta_0_,size=None) 

In [ ]:
print(np.sum(X),np.sum(Z),np.count_nonzero(Z-X))

In [ ]:
# ایده دوم برای مین فیلد و شبه مشاهدات

In [ ]:
# Save true X in Z:
Z=X

In [ ]:
#initialize parameters a0,b0,a1,b1:
A=np.round(np.random.uniform(0, 10, (4,1)),3)
a0,b0,a1,b1=A[0][0],A[1][0],A[2][0],A[3][0]
print(a0,b0,a1,b1)

In [ ]:
def Sample_Sudo_theta0(a0, b0):
    for i in beta.rvs(a0, b0, size=10000):
        if (i>0.001)&(i<0.5):
            theta0=round(i,3)
            break
    return theta0  

In [ ]:
def Sample_Sudo_theta1(a1, b1):
    for i in beta.rvs(a1, b1, size=100000):
        if .9901>i>0.5:
            theta1=round(i,3)
            break
    return theta1  


In [ ]:
theta0=Sample_Sudo_theta0(a0, b0)
theta0

In [ ]:
theta1=Sample_Sudo_theta1(a1, b1)
theta1

In [ ]:
y=np.zeros((n,T))


In [ ]:
def Calculate_Sudo_observations(a0,b0,a1,b1,X):
    for i in range(n):
        j=family_index(i,unique_rows)
        for t in range(T):
            if YF[j,t]==1:
                theta1=Sample_Sudo_theta1(a1+YF[j,t]*np.sum(unique_rows[j])-np.sum(X[unique_rows[j]==1,t])+X[i,t], b1+np.sum(X[unique_rows[j]==1,t])-X[i,t])
                y[i,t]=np.random.binomial( 1, theta1,size=None)
            else:
                y[i,t]=0
    return y

In [ ]:
# Run Forward sampling algorithm to estimate X:
U=10
J=1000
P=p_
X=Forward_Sampling(T,n,G,params)
y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
X=Calculate_X(J,T,n,X,G,y,params)
winsound.Beep(freq, duration)

In [ ]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [ ]:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,y,params)
    params=prm[0]
    print("updated parameters:",params)
    y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
    X=Calculate_X(J,T,n,X,G,y,params)
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)
    
    

In [ ]:
params

In [ ]:
#y=Calculate_Sudo_observations(a0,b0,a1,b1,X)
for i in range(n):
    for t in range(T):
        j=family_index(i,unique_rows)
        if YF[j,t]==1:
            if (np.sum(X[unique_rows[j]==1,t]))==0:
                print(np.sum(X[unique_rows[j]==1,t]))

In [ ]:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]

In [ ]:
np.sum(y)

In [ ]:
print(np.sum(X),np.sum(Z),np.count_nonzero(Z-X))

In [ ]:
# sample parameter of sudo obsarvations:
def Sample_theta(a, b):
    for i in beta.rvs(a, b, size=10000):
        if (i>0.3)&(i<0.6):
            theta=round(i,3)
            break
    return theta 